# 程式說明

作業環境 :`jupyter notebook`

使用語言 :`Python3`

注意:若是顯示錯誤前往更改存取權https://myaccount.google.com/lesssecureapps

### 自訂函式:
- def send_gmail(gmail_addr, gmail_pwd, to_addrs, msg)
   * 與信箱進行互動，登入帳號及寄出訊息
- def get_mime_img(subject, fr, to, img)
   * 將訊息以mime進行傳送
   
### 使用說明:

按下啟動後、登入gmail帳號，並指定傳送的信箱，按下Q結束攝影機。

### 程式介紹:
使用OpenCV將前畫面與現在畫面進行對比，相減後取絕對值，在進行高斯模糊讓他變成灰黑階，轉成黑白像素，最後在找到輪廓後繪製。

In [ ]:
import cv2
import smtplib
from email.mime.image import MIMEImage

#資料設定
gmail_addr = input(str('請輸入帳號 :'))
gmail_pwd =input(str('請輸入密碼 :'))
to_addrs = input(str('請輸入要傳送的信箱 :'))
#函式
def send_gmail(gmail_addr, gmail_pwd, to_addrs, msg):
    smtp_gmail = smtplib.SMTP('smtp.gmail.com', 587)# 建立 SMTP 物件
    print(smtp_gmail.ehlo())# Say Hello
    print(smtp_gmail.starttls())# 啟動 TLS 加密模式
    print(smtp_gmail.login(gmail_addr, gmail_pwd))# 登入
    status = smtp_gmail.sendmail(gmail_addr, to_addrs, msg)# 寄出
    if not status:
        print('寄信成功')
    else:
        print('寄信失敗', status)
    smtp_gmail.quit()# 結束與郵件伺服器的連線
    
#開啟攝影機開始運作
def get_mime_img(subject, fr, to, img):
    #將 img 編碼為 JPGE 格式，[1]為返回資料, [0]為返回是否成功
    img_encode = cv2.imencode('.jpg', img)[1]
    # 再將資料轉為 bytes, 此即為要傳送的資料
    img_bytes = img_encode.tobytes()
    mime_img = MIMEImage(img_bytes)   # 建立 MIMEImage 物件
    mime_img['Content-type'] = 'application/octet-stream'
    mime_img['Content-Disposition'] = 'attachment; filename="pic.jpg"'
    mime_img['Subject'] = subject
    mime_img['From'] = fr
    mime_img['To'] = to
    return mime_img.as_string()   #轉為字串後傳回
cap = cv2.cv2.VideoCapture(0)
skip = 1   # 設定不比對的次數, 由於前影像是空的, 略過一次比對
while cap.isOpened():
    success, img = cap.read()
    if success:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)        # 灰階處理
        img_now = cv2.GaussianBlur(gray, (13, 13), 5)       # 高斯模糊
        if skip > 0:  #如果 skip 大於 0 就略過不和前影像比對
            skip -= 1  #將 skip 次數減 1
        else: #如果 skip==0 就和前影像比對
            diff = cv2.absdiff(img_now, img_pre)   # 此影格與前影格的差異值
            ret, thresh = cv2.threshold(diff, 25, 255,  # 門檻值
                                        cv2.THRESH_BINARY)
            contours, _ = cv2.findContours(thresh,    # 找到輪廓
                                             cv2.RETR_EXTERNAL,
                                             cv2.CHAIN_APPROX_SIMPLE)
            if contours:    # 如果有偵測到輪廓
                cv2.drawContours(img, contours, -1, (255, 255, 255), 2)
                msg = get_mime_img('小偷入侵', '牛奶防盜監視器', '警察局', img)
                send_gmail(gmail_addr, gmail_pwd, to_addrs, msg) #以 gmail 寄出
              
                print('偵測到移動')
                skip = 200    #略過 200 次不比對
            else:
                print('靜止畫面')
        cv2.imshow('frame', img)
        img_pre = img_now.copy()
    k = cv2.waitKey(50)   #暫停 50 毫秒 (0.05 秒), 並檢查是否有按鍵輸入
    if k == ord('q'):
        cv2.destroyAllWindows()
        cap.release()
        break
